In [1]:
import torch

In [4]:
torch.manual_seed(7)

features = torch.randn((1,3))

n_input = features.shape[1]
n_hidden = 2
n_output = 1

weights1 = torch.randn(n_input, n_hidden)
weights2 = torch.randn(n_hidden, n_output)

bias1 = torch.randn((1,n_hidden))
bias2 = torch.randn((1,n_output))

print(features.shape)
print(weights1.shape)
print(bias1.shape)

torch.Size([1, 3])
torch.Size([3, 2])
torch.Size([1, 2])


In [13]:
def activation(score):
    return 1/(1+torch.exp(-score))

In [14]:
def score(X, W, b):
    return torch.mm(X , W.view(features.shape[1],features.shape[0]) )+b

In [17]:
h  = activation(torch.mm(features, weights1) + bias1)
output  = activation(torch.mm(h, weights2) + bias2)
print(output)

tensor([[0.3171]])
